In [12]:
from scamp import test_run, Session, wait
from scamp_extensions import Scale
import random
import time
from pythonosc import udp_client
from config import ARTIFACTS_RELPATH
from scamp_utils import set_default_scamp_settings
from connection.touchosc import TOUCHOSC_CONNECTION_PROPS, test_run
from random import random, choice

set_default_scamp_settings()

ModuleNotFoundError: No module named 'scamp_extensions'

## Using Scamp's osc methods

In [43]:
session = Session().run_as_server()
piano = session.new_osc_part(**TOUCHOSC_CONNECTION_PROPS)


In [32]:
test_run(session, piano, 50, 70, 3, [0, 4], 5)

Running sequence:  [50, 53, 56, 59, 62, 65, 68, 70, 67, 64, 61, 58, 55, 52]
Repeat:  0
Repeat:  1
Repeat:  2
Repeat:  3
Repeat:  4


## Scamp tutorial series

This section starts the scamp tutorial practice once again

In [39]:
session.tempo = 30
for pitch in (48, 53, 67):
    piano.play_note(pitch, 127, 1.0, blocking=False)
    wait(1.0)

In [10]:
session = Session().run_as_server()
piano = session.new_osc_part(**TOUCHOSC_CONNECTION_PROPS)

interval = 1
piano_pitch = 48
forte_piano = 127
diminuendo = 50
low = 36
high = 60

note_choices = [0.1, 0.2, 0.5, 1, 1.5, 2, 3]
rest_choices = [0.1, 0.2, 0.5, 1]
bar_lines = []

session.tempo = 120
while session.time() < 30:
    if random() < 0.7:
        piano.play_note(piano_pitch, forte_piano, choice(note_choices))
        wait(choice(rest_choices))
    else:
        bar_lines.append(session.time())
        piano.play_note(piano_pitch, diminuendo,choice(note_choices))
        wait(choice(rest_choices))
        
    piano_pitch = (piano_pitch + interval - low) % (high - low) + low
    interval += 1
